In [1]:
import pandas as pd
import numpy as np
from pandas import Index
from datetime import datetime
import sys
sys._enablelegacywindowsfsencoding()

In [2]:
%load_ext pycodestyle_magic

In [3]:
def signal_process(table,event_var='PM25_Marseille_Longchamp',event_threshold = 40, duration = 6,var_name_list='air pollution'):
    """Find the spike in pm25 and plot a sliding window for google trend """

    index_list=np.where(table[event_var]>event_threshold)[0]
    
    # add first element
    shrink_index_list = [index_list[0]]
    date_set ={table.date[index_list[0]]}
    
    # exclude for the same day
    for cnt in range(1,len(index_list)):
        if table.date[index_list[cnt]] in date_set:
            pass
        else:
            shrink_index_list.append(index_list[cnt].copy())
            date_set.add(table.date[index_list[cnt]])
    
    bind_list = []
    for start_index in shrink_index_list:
        this_sequence = table.loc[range(start_index-2,start_index+duration),
                                  var_name_list]
        start_date = table.datetime[start_index]
        tuples = list(zip([start_date]*(duration+2),range(-2,duration)))

        this_sequence.index = pd.MultiIndex.from_tuples(tuples,names=['event_date_index','hour_index'])
    #         this_sequence = (this_sequence- this_sequence.mean())/this_sequence.std()

        bind_list.append(this_sequence)
    new_table=pd.concat(bind_list)     
    
    return new_table

In [16]:
parca_table =pd.read_csv("../stage/final_marseille.csv")

In [18]:
# parca_table.head()

In [19]:
# parca_table=pd.read_csv("../stage/parca_difficultés respiratoires_asthme_dyspnée_respiration_PM25_Marseille_Longchamp.csv",encoding='utf-8')

In [5]:
parca_table.columns=Index(['Unnamed: 0', 'datetime', 'PM25_Marseille_Longchamp', 'date',
       'difficultés respiratoires', 'difficultés respiratoires topic', 'isPartial', 'asthme',
       'asthme topic', 'isPartial.1', 'dyspnée', 'dyspnée topic', 'isPartial.2',
       'respiration', 'respiration topic', 'isPartial.3'],
      dtype='object')

In [6]:
keyword_list = ['PM25_Marseille_Longchamp', 'difficultés respiratoires', 'difficultés respiratoires topic', 'asthme',
       'asthme topic', 'dyspnée', 'dyspnée topic',  'respiration', 'respiration topic']

In [7]:
# datetime.strptime('2017-08-31 00:00:00',"%Y-%m-%d %H:%M:%S").date()

In [28]:
parca_table['date']=list(map(lambda stringdate: datetime.strptime(stringdate,"%Y-%m-%d %H:%M:%S").date(),parca_table.datetime))

In [31]:
# parca_tabl

In [32]:
parca_control=parca_table.groupby("date").filter(lambda x: np.all(x['PM25_Marseille_Longchamp']<35))

In [36]:
parca_treatment = parca_table.groupby("date").filter(lambda x: np.any(x['PM25_Marseille_Longchamp']>35))

In [37]:
parca_treatment.to_csv("../stage/parca_treatment.csv",index=False)

In [38]:
parca_treatment.head()

,Unnamed: 0,datetime,PM25_Marseille_Longchamp,difficultés respiratoires,asthme,dyspnée,respiration,banane,banane_topic,la diarrhée,la diarrhée_topic,pollution de l’air,pollution de l’air_topic,date
725,732,2017-09-30 00:00:00,15.0,0,0,0,18,28,38,NaN,NaN,NaN,NaN,2017-09-30
726,733,2017-09-30 01:00:00,23.0,0,0,0,0,14,14,NaN,NaN,NaN,NaN,2017-09-30
727,734,2017-09-30 02:00:00,16.0,0,57,0,29,15,15,NaN,NaN,NaN,NaN,2017-09-30
728,735,2017-09-30 03:00:00,16.0,0,0,0,45,37,49,NaN,NaN,NaN,NaN,2017-09-30
729,736,2017-09-30 04:00:00,15.0,0,24,0,24,19,26,NaN,NaN,NaN,NaN,2017-09-30


In [33]:
parca_control.to_csv('../stage/parca_control.csv',index=False)

In [38]:
new_parca=signal_process(parca_table,event_threshold=35,duration=12,var_name_list=keyword_list)

NameError: name 'signal_process' is not defined

In [16]:
new_parca.to_csv("../output/new_parca.csv")

In [17]:
parif_table = pd.read_csv("../stage/parif_difficultés respiratoires_asthme_dyspnée_respiration_PA04C_PM10.csv")

In [20]:
parif_kw = ['PA04C_PM25',
            'PA04C_PM10',
            'PA04C_O3',
            'PA04C_NO2',
            'PA04C_CO',
            'difficultés respiratoires',
            'asthme',
            'dyspnée',
            'respiration']

In [22]:
new_parif =signal_process(parif_table,event_var='PA04C_PM25',event_threshold =35,duration =12,var_name_list = parif_kw )

In [23]:
new_parif.head()

PA04C_PM25  PA04C_PM10  PA04C_O3  PA04C_NO2  \
event_date_index    hour_index                                                
2018-02-06 19:00:00 -2                35.0        37.0       6.0       54.0   
                    -1                35.0        37.0       5.0       55.0   
                     0                37.0        38.0       4.0       54.0   
                     1                37.0        39.0       5.0       54.0   
                     2                36.0        39.0       3.0       54.0   

                                PA04C_CO  difficultés respiratoires  asthme  \
event_date_index    hour_index                                                
2018-02-06 19:00:00 -2             500.0                          0      19   
                    -1             500.0                          0      18   
                     0             500.0                          0      15   
                     1             600.0                          0      25   
                     2             600.0                          0      17   

                                dyspnée  respiration  
event_date_index    hour_index                        
2018-02-06 19:00:00 -2                6           27  
                    -1                5           32  
                     0                2           28  
                     1                4           33  
                     2                1           39

In [24]:
new_parif.to_csv("../output/new_parif.csv")